In [22]:
import pandas as pd
import numpy as np
import networkx as nx
from shapely.geometry import Point, LineString
from shapely.ops import nearest_points
from geopy.distance import distance, lonlat
from tqdm.notebook import tqdm
import xml.etree.ElementTree as ET
from ast import literal_eval
import re
import pickle


In [ ]:
# Creating the DataFrame of the earthquakes' probabilities
df = pd.read_table('incremental_annual_rates/incremental_annual_rates_USA.txt', sep='\s+', header=None)

df.columns = ['Lat', 'Long'] + [str(np.around(x,1)) for x in np.arange(4.6, 8.7, 0.1)]
df.drop(df.columns[42:], axis=1, inplace=True)
df.fillna(value=0.0, inplace=True)
df = df[df['4.6'] != 0.0]
df.rename(columns={'Lat':'Long', 'Long':'Lat'}, inplace=True)

df.to_csv('incremental_annual_rates/incremental_annual_rates_italy_without_zeros.csv', index=False)

In [ ]:
df = pd.read_csv('earthquake_probabilities/italy.csv')
df.head(2)

In [ ]:
g = nx.read_gml('networks/italy.gml', label='id')

In [ ]:
def intensity_europe(M, R):
    h = 3.91
    D = np.sqrt(h**2 + R**2)
    return 1.621*M - 1.343 - 0.0086*(D-h) - 1.037*(np.log(D)-np.log(h))

In [ ]:
l = [tuple(d.values()) for d in g.edges[('Rome', 'Pescara', 0)]['points']['point']]
edge = LineString(l)
point = Point(10.8,48.3)
nearest_point = nearest_points(edge, point)[0]
nearest_point.coords[:]

In [ ]:
for e in g.edges:
    edge_points = [tuple(p.values()) for p in g.edges[e]['points']['point']]
    edge = LineString(edge_points)
    epicenter = Point(10.5,48.2)
    nearest_point = nearest_points(edge, epicenter)[0].coords[0]
    print(nearest_point)
    print(epicenter.coords[0])
    dist = distance(lonlat(*epicenter.coords[0]), lonlat(*nearest_point)).km
    intensity = intensity_europe(8.6, dist)
    print("%.1f" % intensity)

In [ ]:
#df['Epicenter'] = list(zip(df.Lat, df.Long))
df.insert(loc=0, column='Epicenter', value=list(zip(df.Lat, df.Long)))
df.set_index('Epicenter', inplace=True)
df.drop(['Lat', 'Long'], axis=1, inplace=True)

In [ ]:
df.head()

In [ ]:
epicenters = df.index.to_numpy()
magnitudes = df.columns.to_numpy()

In [ ]:
I = np.ones((g.number_of_edges(), len(epicenters[:2]), len(magnitudes)))
for i, e in tqdm(enumerate(g.edges), total=g.number_of_edges()):
    edge_points = [tuple(p.values()) for p in g.edges[e]['points']['point']]
    l = LineString(edge_points)
    for j, epicenter in tqdm(enumerate(epicenters[:2]), total=len(epicenters)):
        p = Point(epicenter)
        nearest_point = nearest_points(l, p)[0].coords[0]
        R = distance(lonlat(*epicenter), lonlat(*nearest_point)).km
        for k, M in enumerate(magnitudes):
            intensity = intensity_europe(np.float(M), R)
            if intensity > 1.0:
                I[i,j,k] = intensity

In [ ]:
I[I>4]

In [ ]:
def remains_connected(g, srlg):
    g.remove_edges_from(srlg)
    return nx.is_connected(g.to_undirected())

def get_minimal_cut_SRLGs(PSRLG_file)
    xtree = ET.parse(PSRLG_file)
    xroot = xtree.getroot()
    cut_srlgs = []
    for fs in tqdm(xroot.iter('Failure_State'), total=len(xroot.findall('Failure_State'))):
        # (edge_id, node_id, node_id)
        srlg = set([ (int(edge[1]), int(edge[2]), 0) if edge[0] != '24' else (int(edge[1]), int(edge[2]), 1) for edge in re.findall(r"(\d+):\((\d+)\D+(\d+)\D+\)", fs.find('Edges').text)])
        if not remains_connected(g.copy(), srlg):
            added = False
            for i, s in tqdm(enumerate(cut_srlgs), total=len(cut_srlgs)):
                if srlg.issubset(s):
                    cut_srlgs[i] = srlg
                    added = True
                    break
                elif srlg.issuperset(s):
                    added = True
                    break
            if not cut_srlgs or not added:
                cut_srlgs.append(srlg)
        return cut_srlgs

cut_srlgs = get_minimal_cut_SRLGs('PSRLGs/whole_graph_simple_VI_upper_big_grid.xml')

In [23]:
# Write
with open('min_cut_SRLGs/italy', 'wb') as fp:
    pickle.dump(cut_srlgs, fp)

# Read
#with open ('min_cut_SRLGs/italy', 'rb') as fp:
#    cut_srlgs = pickle.load(fp)

In [21]:
cut_srlgs

[{(4, 5, 0), (18, 4, 0)},
 {(14, 13, 0), (15, 14, 0)},
 {(6, 9, 0), (8, 15, 0), (15, 22, 0)},
 {(12, 9, 0), (15, 12, 0)},
 {(0, 7, 0), (0, 24, 0), (1, 0, 0), (20, 0, 0)},
 {(0, 7, 0), (6, 7, 0), (7, 8, 0)},
 {(1, 8, 0), (7, 8, 0), (8, 15, 0), (8, 16, 0)},
 {(18, 2, 0), (18, 3, 0), (18, 4, 0), (24, 18, 0)},
 {(2, 1, 0), (2, 24, 0), (18, 2, 0)},
 {(19, 5, 0), (19, 21, 0), (19, 21, 1)},
 {(0, 7, 0), (1, 0, 0), (1, 8, 0), (6, 20, 0)},
 {(19, 21, 0), (19, 21, 1), (21, 20, 0)},
 {(8, 16, 0), (15, 22, 0)},
 {(15, 22, 0), (22, 16, 0), (22, 17, 0)},
 {(23, 17, 0)},
 {(6, 20, 0), (20, 0, 0), (21, 20, 0)},
 {(0, 7, 0), (1, 8, 0), (6, 7, 0), (7, 8, 0), (8, 16, 0)},
 {(0, 24, 0), (2, 24, 0), (24, 18, 0)},
 {(1, 0, 0), (1, 8, 0), (2, 1, 0)},
 {(8, 16, 0), (16, 17, 0), (22, 16, 0)},
 {(16, 17, 0), (22, 17, 0)},
 {(6, 7, 0), (6, 9, 0), (6, 20, 0)},
 {(6, 7, 0), (6, 9, 0), (6, 20, 0), (7, 8, 0)},
 {(10, 11, 0), (11, 13, 0)},
 {(6, 9, 0), (9, 10, 0), (12, 9, 0)},
 {(9, 10, 0), (10, 11, 0)},
 {(4, 5, 0),